# Investments dashboard

## Import

In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import json
from IPython.display import display
from datetime import datetime, date, timedelta
import os, sys 

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import param

# Own functions
from helpers import load_dict_from_json, make_sure_pd_index_timestamp
from graphs_helpers import get_starting_point, get_range, color_negative_red, highlight_max

import plotly.io as pio

# Interactive panels
import panel as pn
pn.extension('plotly','tabulator')

# Select template
use_template = pio.templates['niinasown']

## Filenames for data

In [12]:
mylibraryPath = 'C:/Users/clima/AnalyticsPortfolio/Investments'

# Data for drawing
draw_data_path = mylibraryPath+'/data/'
draw_data_filename = 'individual_asset_data.csv'
draw_return_filename = 'total_return_data.csv'
draw_kpi_filename = 'kpi_data.json'
draw_continents_distri_filename = 'continents_distri.csv'
draw_country_distri_filename = 'country_distri.csv'
draw_contents_filename = 'contents.csv'
draw_cash_flows_filename_prefix = 'cash_flows_timeseries_'
draw_market_value_filename_prefix = 'market_value_timeseries_'

# Read current assets with details

In [13]:
data = pd.read_csv(draw_data_path+draw_data_filename).set_index('tic')
data.head()

,Name,Type,Volume,Market value,Return eur,3m P/A %,6m P/A %,1y P/A %,3y P/A %,5y P/A %,10y P/A %,max P/A %,Costs % per year,Costs eur,Inv. time y,Latest updated,Sustainability
tic,,,,,,,,,,,,,,,,,
CDE1,Cash deposit1,Deposit,734.55,734,1,-0.000,-0.000,-0.000,0.000,NaN,NaN,0.000,0.0,NaN,3.8,2021-07-19,1
CDE2,Cash deposit2,Deposit,1113.72,1114,-1224,-0.000,-0.000,-0.000,-0.310,-0.277,-0.271,0.093,0.0,0.0,11.3,2023-01-05,1
CDE3,Cash deposit3,Deposit,4643.67,4644,2,-0.000,0.000,-0.000,0.000,NaN,NaN,0.000,0.0,NaN,4.0,2023-01-05,1
COU,Country index ETF,ETF,40.88,2033,-249,0.011,-0.055,-0.124,NaN,NaN,NaN,-0.061,NaN,NaN,1.8,2022-06-17,1
USD,Dollar deposit,Deposit,6091.20,5670,-1592,-0.209,-0.081,0.060,-0.005,0.024,0.019,-0.019,0.0,0.0,10.8,2022-12-20,3


# Read return data and create visualizations to be used by the dashboard
Return data calculated with and without real estate and per asset type

In [14]:
returns = pd.read_csv(draw_data_path+draw_return_filename,sep = ',')
returnFig = go.Figure(data=[go.Table(
    header=dict(values=list(returns.columns),
                align='center',
                fill_color = ['#d8d8d8'],
                font_color = [use_template['layout']['colorway'][0]],
                line_color = [use_template['layout']['colorway'][0]],
                ),
    cells=dict(values=returns.values.T,
               align='center',
               format=["",",.3r",".1%",",.3r",".1%",",.3r",".1%",",.3r",".1%",",.3r",".1%",",.3r",".1%",",.3r",".1%",",.3r"],
               fill_color = [use_template['layout']['colorway'][2]],
               font_color = [use_template['layout']['colorway'][0]],
               line_color = [use_template['layout']['colorway'][0]],             
               ),
    )
    ])

returnFig.update_layout(height=350,
                      width = 1815,
                      margin={"l": 0,"b":0,"r":0,"t":50},
                      title='<b>Return % P/A total and per type during different time periods', 
                      title_x=0.001, 
                      title_font_color=use_template['layout']['colorway'][0],
                      font_size = 14,
                      title_font_size=30
                      ) 
returnFig.update_traces(header_font_size=16, selector=dict(type='table'))
returnFig_pane = pn.pane.Plotly(returnFig)

# Read KPIs and create visualizations to be used by the dashboard

In [15]:
kpis = load_dict_from_json(draw_data_path+draw_kpi_filename)

rows = 1
cols = 5
kpisFig = make_subplots(
    rows=rows,
    cols=cols,
    specs=[[{"type": "indicator"} for c in range(cols)] for t in range(rows)],
    )

kpisFig.add_trace(
                go.Indicator(mode="number", 
                             value=kpis['investmentRate'], 
                             number={"font": {"size": 30}, 'suffix': '%'},
                             title={"text":'Total inv. rate','font_size':20}),
                row=1,
                col=1,
                )

kpisFig.add_trace(
                go.Indicator(mode="number", 
                             value=kpis['investmentRateExclRE'], 
                             number={"font": {"size": 30}, 'suffix': '%'},
                             title={"text":'Total inv. rate excl. real estate','font_size':20}),
                row=1,
                col=2,
                )

kpisFig.add_trace(
                go.Indicator(mode="number", 
                             value=kpis['netMarketValue'], 
                             number={"font": {"size": 30},'suffix':'EUR'},
                             title={"text":'Net market value','font_size':20}),
                row=1,
                col=3,
                )

kpisFig.add_trace(
                go.Indicator(mode="number", 
                             value=kpis['yearlyCostsEur'], 
                             number={"font": {"size": 30},'suffix': " EUR"},
                             title={"text":'Yearly costs','font_size':20,'font_color':use_template['layout']['colorway'][0]}, 
                             ),
                row=1,
                col=4,
                )

kpisFig.add_trace(
                go.Indicator(mode="number", 
                             value=kpis['yearlyCostsPercentage'], 
                             number={"font": {"size": 30}, 'suffix': " %"},
                             title={"text":'Yearly cost','font_size':20,'font_color':use_template['layout']['colorway'][0]},
                             ),
                row=1,
                col=5,
                )

kpisFig.update_layout(height=150,
                      width = 1815,
                      margin={"l": 0,"b":0,"r":0},
                      title='<b>KPIs', 
                      title_x=0.001,           
                      title_font_color=use_template['layout']['colorway'][0],
                      title_font_size=35
                      ) 

kpisFig_pane = pn.pane.Plotly(kpisFig)

# Read Geographical data
Distribution by country and continent data

In [16]:
country_distri = pd.read_csv(draw_data_path+draw_country_distri_filename,sep = ',').set_index('Country')
continent_distri = pd.read_csv(draw_data_path+draw_continents_distri_filename,sep = ',').set_index('Continent')

# Read market value and cash flows time series data

In [17]:
type_graphs_data_values = {}
type_graphs_data_flows = {}
for key in ['Fund', 'ETF', 'Deposit', 'Real estate', 'Totals']:
    temp = pd.read_csv(draw_data_path+draw_market_value_filename_prefix+key+'.csv').set_index('date')
    type_graphs_data_values[key] = make_sure_pd_index_timestamp(temp)
    
for key in ['Fund', 'ETF', 'Deposit', 'Real estate', 'Totals']:
    temp = pd.read_csv(draw_data_path+draw_cash_flows_filename_prefix+key+'.csv').set_index('date')
    type_graphs_data_flows[key] = make_sure_pd_index_timestamp(temp)

# Create Panel Dashboard

In [18]:
from bokeh.models.widgets.tables import NumberFormatter, BooleanFormatter

def get_graphs(Graf_Type,Grafs_start_year,use_template):
    '''Create visualization of cash flows and market values and their totals'''
    
    # Avoid too light colors or too close to black in graphs
    too_light_or_dark = ['#f8ffe5','#cccccc','#5a5766','#48435c']
    usecolors = [x for x in use_template['layout']['colorway'] if x not in too_light_or_dark]
    
    type_flows = type_graphs_data_flows[Graf_Type]
    type_series = type_graphs_data_values[Graf_Type]
    
    start_day = pd.Timestamp(Grafs_start_year,1,1).date()
    type_flows = type_flows.loc[start_day:]
    type_series = type_series.loc[start_day:]
    
    fig = go.Figure()
    # Draw flows and value per tic
    for ind,col in enumerate(type_flows.columns):
        
        tic = col.replace(' cumflows_eur','')
        # Select line color based on tic
        if tic == 'Total':
            linecolor = '#5a5766'
        else:
            linecolor = usecolors[ind]
            
        # Get data range from series
        data_flows, data_value = get_range(type_series, type_flows, tic)
        
        fig.add_trace(
                    go.Scatter(x=data_flows.index, y=data_flows[tic + ' cumflows_eur'],
                            mode='lines',
                            name='Casflow '+tic,
                            line = dict(color = linecolor)
                            )
        )

        fig.add_trace(
                    go.Scatter(x=data_value.index, y=data_value[tic + ' value_eur'],
                            mode='lines',
                            name='Value '+tic,
                            line = dict(color = linecolor)
                            )
        )
    
    fig.update_layout(title='<b>Cash flows and market value',
                      title_font_color=use_template['layout']['colorway'][0],
                      title_font_size=20,
                      width = 1815, height = 800)
    
    fig.update_layout(template = use_template)
    return pn.pane.Plotly(fig, height = 800, width = 1815) 

def types_pie_view(df,use_template):
    '''
    Create a pie using the filtered/non-filtered dataframe
    :return: a Plotly pie wrapped by Panel package
    '''
    fig = px.pie(
                df, 
                values='Market value', 
                names='Type', 
                title='Asset allocation per type'
                )
    fig.update_layout(template = use_template)
    return pn.pane.Plotly(fig, height = 600, width = 900) 

def get_df(asset_type):
    ''' 
    Filter df by asset type
    '''
    subset = data.copy()
    if asset_type == None:
        pass
    else:
        subset = subset.loc[subset.Type == asset_type]
    return subset

def get_table(asset_type):
    '''
    Create a table filtered by asset type
    '''
    subset = get_df(asset_type)
    bokeh_formatters = {
        '3m P/A %': NumberFormatter(format='0.0 %'),
        '6m P/A %' : NumberFormatter(format='0.0 %'),
        '1y P/A %' : NumberFormatter(format='0.0 %'),
        '3y P/A %': NumberFormatter(format='0.0 %'),
        '10y P/A %': NumberFormatter(format='0.0 %'),
        'max P/A %': NumberFormatter(format='0.0 %'),
        'Costs % per year': NumberFormatter(format='0.0 %'),
        'Costs eur': NumberFormatter(format='0'),
        }
    filter_table = pn.widgets.Tabulator(subset, formatters=bokeh_formatters)
    filter_table.style.applymap(color_negative_red).apply(highlight_max)
    return filter_table

def geog_pie_view(By_geography,use_template):
    '''
    Create a pie using the filtered dataframe.
    :return: a Plotly pie wrapped by Panel package
    '''
    if By_geography == 'By country':
        df = country_distri.copy()
    else:
        df = continent_distri.copy()
    fig = px.pie(
                 df, 
                 values=df['Portion %'], 
                 names=df.index.tolist(), 
                 title='Asset allocation by geography',
                 ) 
    fig.update_layout(template = use_template)
    return pn.pane.Plotly(fig, height = 600, width = 900) 

In [19]:
import param 

# To handle interactive selections
class Selections(param.Parameterized):
    
    # select values by which to filter or sort:
    By_geography = param.ObjectSelector(default='By country', objects = ['By country','By continent'], allow_None=False) 
    Graf_Type = param.ObjectSelector(default='ETF', objects = ['Totals','Fund', 'ETF', 'Deposit'] , allow_None = True)  # No real estate
    Grafs_start_year = param.ObjectSelector(default=2012, objects = [x for x in range(2012,2024)], allow_None=False)
    Filtering_current_investments_table = param.ObjectSelector(default=None,objects = [None,'Fund', 'ETF', 'Deposit', 'Real estate'],allow_None=True)
    
    @param.depends('Filtering_current_investments_table')
    def table(self):
        return get_table(self.Filtering_current_investments_table)
    
    @param.depends('By_geography')
    def geog_pie_view(self):
        return geog_pie_view(self.By_geography,use_template)
    
    @param.depends('Graf_Type','Grafs_start_year')
    def get_graphs(self):
        return get_graphs(self.Graf_Type,self.Grafs_start_year,use_template)

In [20]:
# Creating the interactive dashboard
def main(df, country_distri, continent_distri, type_graphs_data_values, type_graphs_data_flows, use_template):
    
    ### Create panes ###
    dash = Selections()
    kpis_panel = pn.Column(kpisFig_pane)
    Returns_panel = pn.Column(returnFig_pane) 
    types_panel = pn.Column(types_pie_view(df,use_template))   
    geog_panel = pn.Column(dash.geog_pie_view)
    tables_panel = pn.Column(pn.pane.Markdown('## Current investments', style={'font-family': "serif"}),  
                             pn.Row(dash.table)                             
                             )
    
    ### Sidebar ###
    # Geographical widget inputs
    geog_widgets_panel = pn.Column(dash.param['By_geography'],
                                   width=120 
                                   )
    
    # Graphs widget inputs
    graphs_widgets_panel = pn.Column(dash.param['Graf_Type'],
                                     dash.param['Grafs_start_year'],
                                     width = 120
                                    )
    
    # Table widget inputs
    table_widgets_panel = pn.Column(dash.param['Filtering_current_investments_table'],
                                    width=120   
                                    )
     
    r1 = pn.Row(kpis_panel)
    r2 = pn.Row(types_panel, geog_panel)
    r3 = pn.Row(Returns_panel)
    r4 = pn.Row(dash.get_graphs)
    r5 = pn.Row(tables_panel)
    
    ### Assembling all layout components ###
    ACCENT_COLOR =  use_template['layout']['colorway'][0] 
    HEADER_FONT_COLOR = use_template['layout']['colorway'][2]
    dashboard = pn.template.FastListTemplate(title="My investments", 
                                             sidebar=pn.Column(pn.Row(pn.Spacer(height=50)),dash.param), 
                                             main=[r1,r2,r3,r4,r5],accent_base_color=ACCENT_COLOR, header_color= HEADER_FONT_COLOR, header_background=ACCENT_COLOR
                                            )
    dashboard.show()
    
if __name__ == '__main__':
    main(data, country_distri, continent_distri, type_graphs_data_values, type_graphs_data_flows, use_template)

Launching server at http://localhost:61052
